In [1]:

import os, sys, time, yaml
os.environ['DISPLAY'] = 'localhost:10.0'
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
from scipy import stats

from collections import deque
from numpy.linalg import inv
import pickle
from functools import partial


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


`pipenv run python -m snakeviz program.prof`

`pipenv run python -m cProfile -o program.prof map_creators2.py`

In [3]:
from utils.one import *

In [4]:
dataset = '/home/alex_odnakov/datasets/kitti_dataset'
sequences_dir = os.path.join(dataset, "sequences")
sequence_folders = [
    f for f in sorted(os.listdir(sequences_dir))
    if os.path.isdir(os.path.join(sequences_dir, f))
]

seq_i=1
i=0
scan_list = {}
labels_list = {}
for i in sequence_folders:
    scan_list[i] = sorted(os.listdir(os.path.join(sequences_dir,i,'velodyne')))
    if os.path.isdir(os.path.join(sequences_dir,i,'labels')):
        labels_list[i] = sorted(os.listdir(os.path.join(sequences_dir,i,'labels')))
i = 400


a = SemLaserScan(color_dict, project=True)

a.open_scan(os.path.join(sequences_dir,sequence_folders[seq_i],'velodyne',scan_list[sequence_folders[seq_i]][i]))
a.open_label(os.path.join(sequences_dir,sequence_folders[seq_i],'labels',labels_list[sequence_folders[seq_i]][i]))
a.colorize()
a.show()

In [5]:
from utils.two import *

In [7]:
second_path = '/home/alex_odnakov/personal/dataset/sequences'
calibration = parse_calibration(os.path.join(second_path,sequence_folders[seq_i], "calib.txt"))
poses = parse_poses(os.path.join(second_path,sequence_folders[seq_i], "poses.txt"), calibration)
poses = np.hstack(poses)
# diffs = []
# for post in posses:
#     diff = np.matmul(inv(pose), past["pose"])

In [ ]:
np.matmul(inv(pose), poses)[:,i*4:i*4+4]

NameError: name 'pose' is not defined

In [ ]:
history = deque()
seq_i = 0
all_labels = []
try:
    for i, bin_name in enumerate(tqdm(scan_list[sequence_folders[seq_i]])):
        scan_filename = os.path.join(sequences_dir,sequence_folders[seq_i], "velodyne", bin_name)
        scan = np.fromfile(scan_filename, dtype=np.float32)

        scan = scan.reshape((-1, 4))

        label_filename = os.path.join(sequences_dir,sequence_folders[seq_i], "labels", os.path.splitext(bin_name)[0] + ".label")
        labels = np.fromfile(label_filename, dtype=np.uint32)
        labels = labels.reshape((-1))
        all_labels.append(labels)
        # convert points to homogenous coordinates (x, y, z, 1)
        points = np.ones((scan.shape))
        points[:, 0:3] = scan[:, 0:3]
        remissions = scan[:, 3]

        pose = poses[i]

        num_concat_points = points.shape[0]
        num_concat_points += sum([past["points"].shape[0] for past in history])
        concated_points = np.zeros((num_concat_points * 4), dtype = np.float32)
        
        start = 0
        concated_points[4 * start:4 * (start + points.shape[0])] = scan.reshape((-1))
        start += points.shape[0]

        for past in history:
            diff = np.matmul(inv(pose), past["pose"])
            tpoints = np.matmul(diff, past["points"].T).T
            tpoints[:, 3] = past["remissions"]
            tpoints = tpoints.reshape((-1))

            concated_points[4 * start:4 * (start + past["points"].shape[0])] = tpoints
            start += past["points"].shape[0]   

        # append current data to history queue.
        history.appendleft({
            "points": points,
            "remissions": remissions,
            "pose": pose.copy()
        })
 
except KeyboardInterrupt:
    print("Loop has been terminated.")
all_labels = np.hstack(all_labels)

  1%|          | 31/4541 [00:01<04:36, 16.30it/s]


Loop has been terminated.


In [ ]:
a = SemLaserScan(color_dict, project=False)
a.set_points(concated_points.reshape((-1,4))[:,:3],concated_poin
             ts.reshape((-1,4))[:,:4])
a.set_label(all_labels)
a.colorize()
# a.show(voxel_size=0.5)
a.save(os.path.join(sequences_dir,sequence_folders[seq_i]),'whole_map',voxel_size=0.1)

(4, 4)

In [ ]:
a = SemLaserScan(color_dict, project=False)
a.set_points(concated_points.reshape((-1,4))[:,:3],concated_poin
             ts.reshape((-1,4))[:,:4])
a.set_label(concated_labels)
a.colorize()
# a.show(voxel_size=0.5)
a.save(os.path.join(sequences_dir,sequence_folders[seq_i]),'whole_map',voxel_size=0.1)

In [ ]:
a = SemLaserScan(color_dict, project=False)
a.open_scan(os.path.join(sequences_dir,sequence_folders[seq_i],'whole_map.bin'))
a.open_label(os.path.join(sequences_dir,sequence_folders[seq_i],'whole_map.label'))
a.colorize()
a.show()

In [ ]:
def load_kitti_scan(file_path):
    scan = np.fromfile(file_path, dtype=np.float32)
    return scan.reshape((-1, 4))

def transform_cloud_to_world_frame(xyz, pose):
    points = np.hstack((xyz, np.ones((xyz.shape[0], 1), dtype=xyz.dtype)))
    transformed_points = pose @ points.T

    return transformed_points.T[:,:3]

def build_kitti_map(base_path):

    # Load KITTI poses
    poses = parse_poses(os.path.join(base_path,"poses.txt"), parse_calibration(os.path.join(base_path,"calib.txt")))
    universal_scan = SemLaserScan(color_dict)
    all_points = []
    all_remissions = []
    all_labels = []
    for idx, pose in enumerate(tqdm(poses)):
        scan_filename = os.path.join(base_path, "velodyne", f'{idx:06d}.bin')
        label_filename = os.path.join(base_path, "labels", f'{idx:06d}.label')
        
        if not os.path.isfile(scan_filename):
            continue
        universal_scan.open_scan(scan_filename)
        universal_scan.open_label(label_filename)
        
        transformed_scan = transform_cloud_to_world_frame(universal_scan.points, pose)
        
        # if idx==0:
        #     all_points = universal_scan.points
        #     all_remissions = universal_scan.remissions
        #     all_labels = universal_scan.sem_label
        #     continue
        
        new_points = transform_cloud_to_world_frame(universal_scan.points, pose)
        new_remissions = universal_scan.remissions
        new_labels = universal_scan.sem_label
        all_points.append(new_points)
        all_remissions.append(new_remissions)
        all_labels.append(new_labels)
        
    all_points = np.vstack(all_points)
    all_remissions = np.hstack(all_remissions)
    all_labels = np.hstack(all_labels)
    universal_scan.set_points(all_points, all_remissions)
    universal_scan.set_label(all_labels)

    return universal_scan

# set your KITTI point cloud directory and poses file
base_path = "/home/aleksadnr/diplom/dataset/sequences/01"

# build the global map
global_cloud = build_kitti_map(base_path)

# visualize the global map



  0%|          | 0/1101 [00:00<?, ?it/s]

100%|██████████| 1101/1101 [00:19<00:00, 57.21it/s]


In [ ]:
import gc
gc.collect()
def show_global_var_sizes():
    global_vars = globals()
    print("{:<20} {:<15} {:<10}".format('Variable', 'Memory Size', 'Type'))
    print("-" * 45)
    
    # Exclude built-in modules and special variables
    excluded_names = set(dir(__builtins__)) | {'In', 'Out', 'get_ipython', 'exit', 'quit'}
    
    for name, obj in global_vars.items():
        if name not in excluded_names and not name.startswith('_'):
            size = sys.getsizeof(obj)
            obj_type = type(obj).__name__
            if size>1024*128:
                print("{:<20} {:<15} {:<10}".format(name, f'{(size//1024)/1024:.2f} MB', obj_type))
show_global_var_sizes()

Variable             Memory Size     Type      
---------------------------------------------
poses                0.55 MB         ndarray   


In [ ]:
global_cloud.colorize()
global_cloud.voxel_ds(voxel_size=0.5)
# global_cloud.show()

: 

In [ ]:
with open('temp.pkl', 'wb') as file: 
    pickle.dump(global_cloud, file) 

In [8]:
with open('temp1.pkl', 'rb') as file: 
    global_cloud  = pickle.load(file) 
global_cloud.colorize()

In [ ]:
global_cloud.show()

In [9]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(global_cloud.points)
pcd.colors = o3d.utility.Vector3dVector(global_cloud.sem_label_color)
downpcd = pcd.voxel_down_sample(voxel_size=0.5)
downpcd

PointCloud with 1620694 points.

In [29]:
global_cloud.sem_label_color.shape

(116355734, 3)

In [ ]:

def voxel_ds(self=global_cloud, voxel_size=None):
    xyz = self.points
    labels = self.sem_label
    remissions = self.remissions
   
    if self.voxel_size is None:
        self.voxel_size = voxel_size
    indices = np.floor(xyz / self.voxel_size).astype(np.int32)
    unique_indices, unique_inverse = np.unique(indices, return_inverse=True, axis=0)
    num_unique_voxels = len(unique_indices)
    self.unique_inverse = unique_inverse
    print('openning loop')
    results = []
    for idx in tqdm(range(num_unique_voxels)):
        results.append(self._downsample(idx))
    downsampled_xyz, downsampled_remissions, downsampled_labels = zip(*results)
    
    self.ds_points = np.vstack(downsampled_xyz)
    self.ds_remissions = np.hstack(downsampled_remissions)

    if len(downsampled_labels)== self.ds_points.shape[0]:
      self.ds_sem_label = np.asarray(downsampled_labels) & 0xFFFF  # semantic label in lower half
      self.ds_inst_label = np.asarray(downsampled_labels) >> 16    # instance id in upper half
    
    self.colorize()

In [ ]:
global_cloud.voxel_ds = voxel_ds

In [ ]:
global_cloud.sem_label.shape

(11635573,)

: 

In [ ]:
a = SemLaserScan(color_dict, project=True)

a.open_scan("/home/aleksadnr/diplom/dataset/sequences/01/whole_map_0.01ds_0.1vs.bin")
a.open_label("/home/aleksadnr/diplom/dataset/sequences/01/whole_map_0.01ds_0.1vs.label")
a.colorize()
a.show(voxel=True,voxel_size=0.1)

In [ ]:
global_cloud.colorize()
print('done colarization')
global_cloud.show(sample_rate=0.1)

done colarization
